In [ ]:
import numpy as np

import tensorflow as tf

!pip install -q tensorflow-hub
!pip install -q tfds-nightly
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

     |████████████████████████████████| 3.3MB 2.8MB/s 
Version:  2.2.0-rc2
Eager mode:  True
Hub version:  0.7.0
GPU is NOT AVAILABLE


In [ ]:
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteAT4XH7/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteAT4XH7/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteAT4XH7/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [ ]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

In [ ]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=50,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/50
30/30 [==============================] - 3s 104ms/step - loss: 0.2049 - accuracy: 0.9224 - val_loss: 0.3119 - val_accuracy: 0.8643
Epoch 2/50
30/30 [==============================] - 3s 105ms/step - loss: 0.1906 - accuracy: 0.9273 - val_loss: 0.3088 - val_accuracy: 0.8655
Epoch 3/50
30/30 [==============================] - 3s 105ms/step - loss: 0.1797 - accuracy: 0.9326 - val_loss: 0.3074 - val_accuracy: 0.8646
Epoch 4/50
30/30 [==============================] - 3s 104ms/step - loss: 0.1698 - accuracy: 0.9385 - val_loss: 0.3074 - val_accuracy: 0.8689
Epoch 5/50
30/30 [==============================] - 3s 103ms/step - loss: 0.1585 - accuracy: 0.9424 - val_loss: 0.3058 - val_accuracy: 0.8667
Epoch 6/50
30/30 [==============================] - 3s 110ms/step - loss: 0.1506 - accuracy: 0.9477 - val_loss: 0.3064 - val_accuracy: 0.8667
Epoch 7/50
30/30 [==============================] - 3s 108ms/step - loss: 0.1420 - accuracy: 0.9521 - val_loss: 0.3075 - val_accuracy: 0.8668
Epoch 

In [ ]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

49/49 - 3s - loss: 0.6277 - accuracy: 0.8416
loss: 0.628
accuracy: 0.842


In [ ]:
print("Enter No. of ratings\n")
total = int(input())
print("\n")

count = 0
for i in range (int(total)):     
    a = input("Enter a review: ")
    result = model.predict_classes([a])
    if(result[0][0] == 1):
      print("Positive review\n")
    else:
      print("Negative review\n")
   
    if(result[0][0]):
      count +=1
print()

x = (count/total)* 5
print("Movie Rating : %.1f / 5" % (x))

Enter No. of ratings

3


Enter a review: good movie
Positive review

Enter a review: great
Positive review

Enter a review: bad movie
Negative review


Movie Rating : 3.3 / 5
